<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

## Contexto

**Descripción:**

SetMagic Productions es una empresa especializada en la provisión de servicios integrales para la realización de rodajes cinematográficos y audiovisuales. Nos dedicamos a facilitar tanto el atrezzo necesario para las producciones como los lugares idóneos para llevar a cabo los rodajes, ya sea en entornos al aire libre o en interiores.

**Servicios Ofrecidos:**

- **Atrezzo Creativo:** Contamos con un extenso catálogo de atrezzo que abarca desde accesorios hasta muebles y objetos temáticos para ambientar cualquier tipo de  escena.

- **Locaciones Únicas:** Nuestra empresa ofrece una amplia selección de locaciones, que incluyen desde escenarios naturales como playas, bosques y montañas, hasta espacios interiores como estudios, casas históricas y edificios emblemáticos.
- **Servicios de Producción:** Además de proporcionar atrezzo y locaciones, también ofrecemos servicios de producción audiovisual, incluyendo equipos de filmación, personal técnico y servicios de postproducción.

**Herramientas y Tecnologías:**

Para recopilar información sobre nuevas locaciones y tendencias en atrezzo, utilizamos herramientas de web scraping como Beautiful Soup y Selenium para extraer datos de sitios web relevantes y redes sociales especializadas en cine y producción audiovisual. También integramos APIs de plataformas de alquiler de locaciones y bases de datos de atrezzo para acceder a información actualizada y detallada.

**Almacenamiento de Datos:** (A trabajar la próxima semana)

La información recopilada mediante web scraping y APIs se almacenará tanto en una base de datos relacional SQL como en una base de datos no relacional MongoDB . Estas base de datos nos permite organizar eficientemente la información sobre locaciones, atrezzo, clientes y proyectos en curso, facilitando su acceso y gestión.

**Objetivo:**

Nuestro objetivo principal es proporcionar a nuestros clientes una experiencia fluida y personalizada en la búsqueda y selección de locaciones y atrezzo para sus proyectos audiovisuales. Utilizando tecnologías avanzadas y una amplia red de contactos en la industria, nos esforzamos por ofrecer soluciones creativas y de alta calidad que satisfagan las necesidades específicas de cada producción.


## Lab: Extracción de Información con Beautiful Soup

En este laboratorio seguirás enriqueciendo la base de datos para ofrecer a tus clientes un servicio más completo y personalizado. Para lograr esto, extraerás información valiosa sobre objetos de atrezzo de una web especializada. Utilizarás técnicas de web scraping con la biblioteca **Beautiful Soup** para obtener y organizar los datos de manera eficiente.

Trabajarás con la siguiente URL: [Atrezzo Vázquez](https://atrezzovazquez.es/shop.php?search_type=-1&search_terms=&limit=48&page=1). Esta página contiene una gran cantidad de objetos de atrezzo que pueden ser de interés para un proyecto de rodaje. Tu tarea será extraer información relevante de los productos listados en las primeras 100 páginas.

1. **Navegación y Extracción de Múltiples Páginas**:

   - La página web contiene varios elementos distribuidos en distintas páginas. Tu objetivo será iterar a través de las 100 primeras páginas y extraer la información deseada.

   - Debes asegurarte de que tu código sea capaz de navegar automáticamente por estas páginas y extraer datos de manera continua.

2. **Verificación del Código de Estado de la Respuesta**:

   - Antes de extraer cualquier información, es fundamental verificar que la solicitud a la página web ha sido exitosa. Un código de estado 200 indica que la página se ha cargado correctamente.

   - Si el código no es 200, debes imprimir un mensaje de error y detener la ejecución de la extracción para evitar problemas posteriores.

3. **Extracción de Información Específica**:

   De cada página, deberás extraer los siguientes detalles de los objetos de atrezzo:

   - **Nombre del Objeto**: El nombre o identificador del objeto.

   - **Categoría**: La categoría en la que se clasifica el objeto (ej.: mobiliario, decorado, utilería, etc.).

   - **Sección**: La sección específica dentro de la categoría.

   - **Descripción**: Una breve descripción del objeto que puede incluir detalles sobre su estilo, material o uso.

   - **Dimensiones**: El tamaño del objeto en formato (largo x ancho x alto).

   - **Enlace a la Imagen**: El link a la imagen del objeto, útil para tener una vista previa visual.

4. **Organización de los Datos en un Diccionario**:

   Una vez extraída la información, deberás organizarla en un diccionario con las siguientes claves:

   - `"nombre"`: Nombres del objeto.

   - `"categoria"`: Categoría a la que pertenece el objeto.

   - `"seccion"`: Sección específica dentro de la categoría.

   - `"descripcion"`: Breve descripción del objeto.

   - `"dimensiones"`: Dimensiones del objeto en el formato adecuado.

   - `"imagen"`: URL de la imagen del objeto.

5. **Almacenamiento de la Información en un DataFrame**:

   Una vez que tengas toda la información organizada en diccionarios, el siguiente paso es convertirla en un DataFrame de Pandas. Este DataFrame te permitirá manipular y analizar los datos con facilidad. Tu DataFrame debería verse similar al ejemplo proporcionado, donde cada fila representa un objeto diferente y cada columna contiene la información extraída:

![Dataframe](https://github.com/Hack-io-Data/Imagenes/blob/main/02-Imagenes/BS/df_atrezzo.png?raw=true)


6.  Consideraciones Adicionales:

- Asegúrate de manejar posibles errores o excepciones durante el scraping, como tiempos de espera agotados, páginas inaccesibles o datos faltantes.

- Recuerda que el scraping debe hacerse de manera respetuosa, evitando sobrecargar el servidor de la página web (puedes usar pausas entre solicitudes).

- Finalmente, asegúrate de almacenar el DataFrame en un archivo CSV para que puedas reutilizar esta información en análisis futuros.


In [2]:
# Importamos las librerías que necesitamos
import numpy as np
# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import re
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
from tqdm import tqdm
import random
import time
import itertools

In [3]:
#funcion buena
def web(sopa,df):

    lista_info = sopa.findAll('div', {'class': "col-md-3 col-sm-4 shop-grid-item"})

    descripciones= [element.find('p').getText() for element in lista_info]

    # Sacar nombres
    lista_nombres = sopa.findAll('a', {'class': 'title'})
    nombres_objetos = [nombre.getText() for nombre in lista_nombres]
    # Sacar categorías
    lista_categorias = sopa.findAll('div', {'class': 'cat-sec-box'})
    nombre_categorias = [ncat.getText().replace('\xa0\xa0','') for ncat in lista_categorias]
    # Sección
    lista_secciones_if = []
    for elemento in lista_info:
        try:
            lista_secciones_if.append(elemento.find('a', {'class':'tag'}).text)
        except:
            lista_secciones_if.append(np.nan)

    # print(len(lista_secciones_if))
    # lista_secciones = sopa_atrezzo.findAll('a', {'class': 'tag'})
    # nombre_secciones = [nsec.getText() for nsec in lista_secciones]

    # Descripción

    lista_img = []
    for elemento in lista_info:
        try:
            lista_img.append(elemento.find('img')['src'])
        except:
            lista_img.append(np.nan)
    # print(len(lista_img))
    # Enlace a imagen
    # lista_imagen = sopa_atrezzo.findAll('img')
    # lista_imagen = [link.get('src') for link in lista_imagen]

    lista_dimensiones= sopa.findAll('div', {'class': 'price'})
    nombre_dimensiones = [dimensiones.getText().strip() for dimensiones in lista_dimensiones]
    # for i in nombre_dimensiones:
    #     i.getText().strip()
    diccionario = {'nombre': nombres_objetos, 'categoria': nombre_categorias, 'seccion': lista_secciones_if, 'descripcion': descripciones, 'dimensiones': nombre_dimensiones, 'imagen': lista_img}
    df_articulos = pd.DataFrame(diccionario)
    df = pd.concat([df, df_articulos])
    return df


In [12]:
#funcion final prueba
df_lleno = pd.DataFrame()
df_final = pd.DataFrame()
for i in range(1,101):
    url_atrezzo = f'https://atrezzovazquez.es/shop.php?search_type=-1&search_terms=&limit=48&page={i}'
    # Paso 2
    response = requests.get(url_atrezzo)
    # Paso 3
    if str(response) == '<Response [200]>':
        print(response.status_code)
        # Paso 4
        sopa_atrezzo = (BeautifulSoup(response.content, 'html.parser'))
        df_final = web(sopa_atrezzo, df_final)
        df_lleno = pd.concat([df_lleno, df_final])
        time.sleep(random.uniform(1,3))
    else: 
        print('Hay un error')
        break


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [13]:
df_final.shape

(4800, 6)

In [22]:
df_final.head()

,nombre,categoria,seccion,descripcion,dimensiones,imagen
0,ALF1,Arabe,Alfombra,Alfombra persa marrón (tiene unas manchas que ...,460x340x1 (cm),admin/img_prod/ALF1/ALF1-720x540.png
1,ADO1,,Adornos,Vitrina con abanico,100x10x60 (cm),admin/img_prod/ADO1/ADO1-720x540.png
2,AFC1,Dormitorio,Alfombra de cama,Alfombrín de cama círculos azules,150x72x1 (cm),admin/img_prod/AFC1/AFC1-720x540.png
3,AP1,AlemánComedor,Aparador,Aparador alemán 2 cuerpos,207x64x300 (cm),admin/img_prod/AP1/AP1-720x540.png
4,AST1,Astronomía,Atrezzo astronomia,Pie de madera para astrolabio,78x78x130 (cm),admin/img_prod/AST1/AST1-pie-720x540.png


A partir de este, saqué la funcion

In [ ]:
# prueba para uno
url_atrezzo = f'https://atrezzovazquez.es/shop.php?search_type=-1&search_terms=&limit=48&page=1'
# Paso 2
response = requests.get(url_atrezzo)
# Paso 3
if str(response) == '<Response [200]>':
    print(response.status_code)
    # Paso 4
    sopa_atrezzo = (BeautifulSoup(response.content, 'html.parser'))
    time.sleep(random.uniform(1,3))
else: 
    print('Hay un error')
    


200


In [ ]:
lista_info = sopa_atrezzo.findAll('div', {'class': "col-md-3 col-sm-4 shop-grid-item"})

descripciones= [element.find('p').getText() for element in lista_info]

# Sacar nombres
lista_nombres = sopa_atrezzo.findAll('a', {'class': 'title'})
nombres_objetos = [nombre.getText() for nombre in lista_nombres]
# Sacar categorías
lista_categorias = sopa_atrezzo.findAll('div', {'class': 'cat-sec-box'})
nombre_categorias = [ncat.getText().replace('\xa0\xa0','') for ncat in lista_categorias]
# Sección
lista_secciones_if = []
for elemento in tqdm(lista_info):
    try:
        lista_secciones_if.append(elemento.find('a', {'class':'tag'}).text)
    except:
        lista_secciones_if.append(np.nan)

print(len(lista_secciones_if))
# lista_secciones = sopa_atrezzo.findAll('a', {'class': 'tag'})
# nombre_secciones = [nsec.getText() for nsec in lista_secciones]

# Descripción

lista_img = []
for elemento in tqdm(lista_info):
    lista_img.append(elemento.find('img')['src'])
print(len(lista_img))
# Enlace a imagen
# lista_imagen = sopa_atrezzo.findAll('img')
# lista_imagen = [link.get('src') for link in lista_imagen]

lista_dimensiones= sopa_atrezzo.findAll('div', {'class': 'price'})
nombre_dimensiones = [dimensiones.getText().strip() for dimensiones in lista_dimensiones]

diccionario = {'nombre': nombres_objetos, 'categoria': nombre_categorias, 'seccion': lista_secciones_if, 'descripcion': descripciones, 'dimensiones': nombre_dimensiones, 'imagen': lista_img}
df_articulos = pd.DataFrame(diccionario)
df = pd.DataFrame()
df = pd.concat([df, df_articulos])

100%|██████████| 48/48 [00:00<00:00, 16017.71it/s]


48


100%|██████████| 48/48 [00:00<00:00, 23998.88it/s]

48


In [ ]:
df.shape

(48, 6)

In [ ]:
print(len(nombre_dimensiones))
print(len(lista_img))
print(len(nombre_dimensiones))
print(len(lista_secciones_if))
print(len(nombre_categorias))
print(len(nombres_objetos))
print(len(descripciones))

48
48
48
48
48
48
48


In [ ]:
df.head()